labels.npy corrisponde alla struttura che contiene le info di classe, oggetto e posizione sulla griglia. Nel caso del codice che ho su github, 

 0 = row_id, 1 = col_id, 2 = class_id, colonna 3 = object_id

In [1]:
import numpy as np
lut = np.load('SITS-Missing-Data/D3_balaruc_lut.npy')
masks = np.load('SITS-Missing-Data/D2_balaruc_masks.npy')

In [6]:
a = masks.flatten()

In [19]:
np.where(a == 1)
a.shape, np.count_nonzero(a)
#np.count_nonzero(a)
masks.shape
#np.count_nonzero(masks, keepdims=True)

(137606, 54)

In [2]:
lut.shape
# 0: IdObject, 1: Class, 2: ?, 3: ?

(137606, 4)

In [3]:
clID_col = lut[:, 1]
clID = np.unique(clID_col)
hashClID2obj = {}
for val in clID:
        idx = np.where(clID_col == val)
        t_labels = lut[idx]
        hashClID2obj[val] = np.unique( t_labels[:,0] )

In [5]:
hashClID2obj.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [7]:
hashClID2obj[1]

array([ 711,  712,  713,  714,  715,  716,  717,  718,  719,  720,  721,
        722,  723,  724,  725,  726,  727,  728,  729,  730,  731,  732,
        733,  734,  735,  736,  737,  738,  739,  740,  741,  742,  743,
        744,  745,  746,  747,  748,  749,  750,  751,  752,  753,  754,
        755,  756,  757,  758,  759,  760,  761,  762,  763,  764,  765,
        766,  767,  768,  769,  770,  771,  772,  773,  774,  775,  776,
        777,  778,  779,  780,  781,  782,  783,  784,  785,  786,  787,
        788,  789,  790,  791,  792,  793,  794,  795,  796,  797,  798,
        799,  800,  801,  802,  803,  804,  805,  806,  807,  808,  809,
        810,  811,  812,  813,  814,  815,  816,  817,  818,  819,  820,
        821,  822,  823,  824,  825,  826,  827,  828,  829,  830,  831,
        832,  833,  834,  835,  836,  837,  838,  839,  840,  841,  842,
        843,  844,  845,  846,  847,  848,  849,  850,  851,  852,  853,
        854,  855,  856,  857,  858,  859,  860,  8

In [1]:
import utils

2022-03-01 18:09:30.166785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-01 18:09:30.166822: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
training, validation, test, num_classes, num_steps, num_bands = utils.load_sits()

In [9]:
training.cardinality(), validation.cardinality(), test.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=59764>,
 <tf.Tensor: shape=(), dtype=int64, numpy=49175>,
 <tf.Tensor: shape=(), dtype=int64, numpy=28667>)

In [5]:
training.cardinality(), validation.cardinality(), test.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=56336>,
 <tf.Tensor: shape=(), dtype=int64, numpy=32095>,
 <tf.Tensor: shape=(), dtype=int64, numpy=49175>)